In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files

import zipfile
import os

uploaded = files.upload()
for filename in uploaded.keys():
    # Open the ZIP file
    with zipfile.ZipFile(filename, 'r') as z:
        csv_file_name = z.namelist()[0]  # Update this if the CSV is not the first file
        # Extract the CSV file to a temporary location
        z.extract(csv_file_name)
dataset = pd.read_csv(csv_file_name, names=['id', 'title', 'text', 'label'])

print(dataset.head(5))


In [ ]:
import seaborn as sns

categoricalDataset=dataset.select_dtypes(include='object')
names=[]
for col in categoricalDataset:
  names.append(col)
uniqueVals=[]
for i in names:
  uniqueVals.append(dataset[i].unique().size)
sns.barplot(x=names,y=uniqueVals)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False)
encoded_data=encoder.fit_transform(dataset[['label']])
dataset=pd.concat([dataset,pd.DataFrame(encoded_data,columns=encoder.get_feature_names_out(['label']))], axis=1)
dataset.drop(columns=['label'], inplace=True)



In [ ]:
print(dataset.head())

In [ ]:
labels=dataset.pop('label_REAL')

In [ ]:
from sklearn.model_selection import train_test_split
dataset.drop(columns=['label_label'],inplace=True)
dataset.drop(columns=['label_FAKE'],inplace=True)


In [ ]:
train_data, test_data, train_labels, test_labels=train_test_split(dataset,labels,test_size=0.2,random_state=20)
print(test_labels)

In [ ]:
from sklearn.pipeline import make_pipeline
model=make_pipeline(TfidfVectorizer(),PassiveAggressiveClassifier())
model.fit(train_data['text'],train_labels)

the real and fake labels need to be hot encoded or mapped to ones and zeros.  initialize the vectorizer and then fit_transform the data for the coloumn.  Then train and fit using the passive agressive classifier

In [ ]:
predictions=model.predict(test_data['text'])
print(predictions)

In [ ]:
correct=0
total=0
final_values=test_labels.tolist()
for i in range(len(test_labels)):
  if final_values[i]==predictions[i]:
    correct+=1
  total+=1
print(correct/total)